# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/acoullandreau/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [18]:
# import libraries

#note that sklearn's version should be at least 0.20.0

#import for NLP
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer

#import for object manipulation
import numpy as np
import pandas as pd
import re
from sqlalchemy import create_engine

#import for ML pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report

In [30]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("SELECT * FROM MessagesWithCategory", engine)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
#we define X as the first 4 columns (we will filter out the 'messages' column afterwards) and the other
#36 category columns as Y
X = df.iloc[:, :4]
Y = df.iloc[:, 5:]

### 2. Write a tokenization function to process your text data

In [32]:
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    #we convert the text to lower case
    text = text.lower()
    
    #we remove any url contained in the text
    
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    url_in_msg = re.findall(url_regex, text)
    for url in url_in_msg:
        text = text.replace(url, "urlplaceholder")
        
    # we remove the punctuation
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # we tokenize the text
    words = word_tokenize(text)
    
    # we lemmatize  and remove the stop words
    words = [lemmatizer.lemmatize(word) for word in words if word not in stopwords.words('english')]
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(random_state=42), n_jobs=1))
])
#for the MultiOutputClassifier, we initially set n_jobs to -1 (which means all processors should be used
#to compute the function), but this cause GridSearchCV to fail - so n_jobs set back to 1.


Notes on the choices:
- The classifier used is Random Forest with its default parameters. This is the classifier chosen for 3 reasons:
    - there are many "features" (i.e words in our case) to consider, and the model will focus on the important variables for the classification
    - the dataset we are looking at is not too big
    - the risk of overfitting the data is low
- CountVectorizer is instantiated using the tokenize() function defined previously - first the input is tokenized then vectorized
- Associated to this vectorisation step, we apply TF-IDF on the matrix obtained as to take into account the frequency of each word in all messages. This allows to weigh each word relatively to how often it appears in all messages, thus understanding the importance of each word for the assignment to the categories labelling a message.


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [33]:
#we split the column 'message' of X and the whole Y dataframe into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X['message'], Y, test_size=0.2, random_state=42)

#we fit the pipeline using the training sets
pipeline.fit(X_train, y_train)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...
            oob_score=False, random_state=42, verbose=0, warm_start=False),
           n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [34]:
#we predict the categories using the testing set
y_pred = pipeline.predict(X_test)

In [35]:
#we define a function to build the classification report on each category of Y
def compute_metrics(y_test, y_pred):
    i=0
    reports = {}
    for column in y_test.columns:
        report = classification_report(y_test[column], y_pred[:,i], labels=np.unique(y_pred[:,i]), output_dict=True)
        reports[column]=report
        i+=1

    for report in reports:
        print('Classification report for "{}" category'.format(report))
        print('     F1 score (avg):', reports[report]['weighted avg']['f1-score'])
        print('     Precision (avg):', reports[report]['weighted avg']['precision'])
        print('     Recall (avg):', reports[report]['weighted avg']['recall'])
        print('\n')
        
compute_metrics(y_test, y_pred)

Classification report for "request" category
     F1 score (avg): 0.8747370913228769
     Precision (avg): 0.8813255531042253
     Recall (avg): 0.8880625476735317


Classification report for "offer" category
     F1 score (avg): 0.9975148155228445
     Precision (avg): 0.9950419527078566
     Recall (avg): 1.0


Classification report for "aid_related" category
     F1 score (avg): 0.745991971708956
     Precision (avg): 0.7489296754303919
     Recall (avg): 0.7507627765064836


Classification report for "medical_help" category
     F1 score (avg): 0.8958845154455399
     Precision (avg): 0.8982421575096262
     Recall (avg): 0.9216247139588101


Classification report for "medical_products" category
     F1 score (avg): 0.9347878873389496
     Precision (avg): 0.9428804172392563
     Recall (avg): 0.9521357742181541


Classification report for "search_and_rescue" category
     F1 score (avg): 0.9667751437808402
     Precision (avg): 0.972250695811966
     Recall (avg): 0.97673531655225

Main conclusions at this point:
- all three metrics are homogeneous for each category (i.e they are centered more or less around the same value for each category individually)
- in general, the scores are between 0.74 and 1
- the lowest scores are obtained for the following categories:
    - aid_related
    - request
    - other_aid
    - direct_report
- the highest scores are obtained for the follosing categories:
    - offer
    - child_alone
    - tools
    - hospitals
    - shops
    - aid_centers

We should focus in increasing the scores of the categories with the lowest scores, ideally having all scores ranging between 0.9 and 1.

### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
#Let's first take a look at the current parameters set for the pipeline
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a37159d90>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
               oob_score=False, random_state=

In [ ]:
#let's set a list of parameters that have an influence on all estimators

parameters = {
    'vect__ngram_range':[(1, 1), (1, 2)],
    'vect__max_df':[0.25, 0.5, 0.75, 1],
    'vect__max_features':[None, 5000, 10000],
    ##'vect__min_df':[1, 2, 4],
    ##'vect__binary': [True, False],
    ##'clf__estimator__bootstrap':[True, False],
    'clf__estimator__min_samples_leaf': [1, 2, 4],
    'clf__estimator__min_samples_split': [2, 3, 4],
    'clf__estimator__max_depth': [None, 10, 20, 50],
    'clf__estimator__max_features': ['auto', 'log2'],
    'clf__estimator__n_estimators':[10, 100, 250]
}
            
cv = GridSearchCV(pipeline, param_grid=parameters, cv=3) 
#we specify cv=3, i.e the cross-validation splitting strategy - 3 folds 

#let's repeat the fit and predict steps but now trying with all combinations of parameters set above
cv.fit(X_train, y_train)
y_pred_optim = cv.predict(X_test)


It seems like the optimal combination of parameters differs from the default set on:
- dddd
- dddd

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
#we simply run the function defined above with the new y_pred
compute_metrics(y_test, y_pred)

We can observe that with the optimal set of parameters found ealier with GridSearch, we now have the scores ranging between and 1. Which is an improvement! Can we do better?

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.